In [111]:
import pyspark
from pyspark.sql import SparkSession

In [112]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

In [113]:
df_green = spark.read.parquet('../data/pq/green/*/*')

In [114]:
"""
SELECT 
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM 
    green
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY 
    1,2
ORDER BY
    1,2
"""

"\nSELECT \n    date_trunc('hour', lpep_pickup_datetime) AS hour, \n    PULocationID AS zone,\n\n    SUM(total_amount) AS amount,\n    COUNT(1) AS number_records\nFROM \n    green\nWHERE\n    lpep_pickup_datetime >= '2020-01-01 00:00:00'\nGROUP BY \n    1,2\nORDER BY\n    1,2\n"

In [115]:
df_green.show()

+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|VendorID|lpep_pickup_datetime|lpep_dropoff_datetime|store_and_fwd_flag|RatecodeID|PULocationID|DOLocationID|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|ehail_fee|improvement_surcharge|total_amount|payment_type|trip_type|congestion_surcharge|
+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|       2| 2020-01-16 23:31:44|  2020-01-16 23:46:21|                 N|         1|          65|          56|              1|          8.6|       24.5|  0.5|    0.

In [116]:
df_green.rdd

MapPartitionsRDD[186] at javaToPython at NativeMethodAccessorImpl.java:0

In [117]:
df_green.rdd.take(10)

[Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 1, 16, 23, 31, 44), lpep_dropoff_datetime=datetime.datetime(2020, 1, 16, 23, 46, 21), store_and_fwd_flag='N', RatecodeID=1, PULocationID=65, DOLocationID=56, passenger_count=1, trip_distance=8.6, fare_amount=24.5, extra=0.5, mta_tax=0.5, tip_amount=0.0, tolls_amount=0.0, ehail_fee=None, improvement_surcharge=0.3, total_amount=25.8, payment_type=2, trip_type=1, congestion_surcharge=0.0),
 Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 1, 14, 15, 1, 42), lpep_dropoff_datetime=datetime.datetime(2020, 1, 14, 15, 28, 14), store_and_fwd_flag='N', RatecodeID=1, PULocationID=129, DOLocationID=141, passenger_count=5, trip_distance=5.16, fare_amount=20.5, extra=0.0, mta_tax=0.5, tip_amount=0.0, tolls_amount=0.0, ehail_fee=None, improvement_surcharge=0.3, total_amount=24.05, payment_type=2, trip_type=1, congestion_surcharge=2.75),
 Row(VendorID=None, lpep_pickup_datetime=datetime.datetime(2020, 1, 24, 5, 55), lpep_dropoff

In [118]:
rdd = df_green \
        .select('lpep_pickup_datetime', 'PULocationID', 'total_amount') \
        .rdd

In [119]:
rdd.take(10)

[Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 16, 23, 31, 44), PULocationID=65, total_amount=25.8),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 14, 15, 1, 42), PULocationID=129, total_amount=24.05),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 24, 5, 55), PULocationID=242, total_amount=62.87),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 31, 8, 19, 3), PULocationID=166, total_amount=8.76),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 12, 18, 54), PULocationID=41, total_amount=14.54),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 11, 17, 13, 41), PULocationID=66, total_amount=18.36),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 29, 19, 23, 3), PULocationID=260, total_amount=8.8),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 26, 11, 45, 13), PULocationID=116, total_amount=4.3),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 3, 8, 58, 57), PULocationID=74, total_amount=7.8),
 Row(lpep_pickup_datetime=datetime.datetime(20

In [120]:
from datetime import datetime

In [121]:
start = datetime(year=2020, month=1, day=1)

In [122]:
def filter_outliers(row):
    return row.lpep_pickup_datetime >= start

In [123]:
rows = rdd.take(10)
row = rows[0]

In [124]:
row.lpep_pickup_datetime.replace(minute=0, second=0, microsecond=0)

datetime.datetime(2020, 1, 16, 23, 0)

In [125]:
def prepare_for_grouping(row):
    hour = row.lpep_pickup_datetime.replace(minute=0, second=0, microsecond=0)
    zone = row.PULocationID
    key = (hour, zone)
    
    amount = row.total_amount
    count = 1
    value = (amount, count)
    return (key, value)

In [126]:
def calculate_revenue(left_value, right_value):
    left_amount, left_count = left_value
    right_amount, right_count = right_value
    
    output_amount = left_amount + right_amount
    output_count = left_count + right_count
    return (output_amount, output_count)

In [127]:
from collections import namedtuple

In [128]:
RevenueRow = namedtuple('RevenueRow', ['hour', 'zone', 'revenue', 'count'])

In [129]:
def unwrap(row):
    return RevenueRow(
        hour=row[0][0],
        zone=row[0][1],
        revenue=row[1][0],
        count=row[1][1])

In [130]:
from pyspark.sql import types

In [131]:
result_schema = types.StructType([
    types.StructField('hour', types.TimestampType(), True), 
    types.StructField('zone', types.IntegerType(), True), 
    types.StructField('revenue', types.DoubleType(), True), 
    types.StructField('count', types.IntegerType(), True)
])

In [132]:
#rdd.filter(lambda row: row.lpep_pickup_datetime >= start).take(1)

df_result = rdd.filter(filter_outliers) \
                .map(prepare_for_grouping) \
                .reduceByKey(reduce) \
                .map(unwrap) \
                .toDF(result_schema)

In [133]:
df_result.schema

StructType([StructField('hour', TimestampType(), True), StructField('zone', IntegerType(), True), StructField('revenue', DoubleType(), True), StructField('count', IntegerType(), True)])

In [134]:
df_result.show()

+-------------------+----+------------------+-----+
|               hour|zone|           revenue|count|
+-------------------+----+------------------+-----+
|2020-01-31 08:00:00| 166| 487.2900000000001|   28|
|2020-01-12 18:00:00|  41| 916.7399999999996|   66|
|2020-01-17 11:00:00| 116|233.65000000000003|   14|
|2020-01-27 18:00:00|  75|1256.4599999999998|   86|
|2020-01-09 09:00:00|  35|342.08000000000004|   14|
|2020-01-16 06:00:00|  42|            337.77|   14|
|2020-01-05 07:00:00|  76|             62.56|    4|
|2020-01-04 23:00:00| 205|              58.8|    1|
|2020-01-15 19:00:00|  61|197.76000000000005|    9|
|2020-01-30 11:00:00|  25| 290.3300000000001|   16|
|2020-01-30 19:00:00| 255|186.77999999999997|   14|
|2020-01-14 20:00:00|  41| 624.4899999999998|   43|
|2020-01-24 16:00:00|  95| 636.4799999999997|   52|
|2020-01-13 13:00:00|  41| 589.3199999999998|   45|
|2020-01-12 20:00:00|  42| 333.5700000000001|   22|
|2020-01-16 09:00:00|  74|1392.6799999999987|   98|
|2020-01-12 

In [136]:
df_result.write.parquet('tmp/green-revenue', mode='overwrite')

In [137]:
columns = ['VendorID', 'lpep_pickup_datetime', 'PULocationID', 'DOLocationID', 'trip_distance']

duration_rdd = df_green \
    .select(columns) \
    .rdd

In [138]:
import pandas as pd

In [139]:
rows = duration_rdd.take(10)

In [140]:
df = pd.DataFrame(rows, columns=columns)

In [105]:
columns

['VendorID',
 'lpep_pickup_datetime',
 'PULocationID',
 'DOLocationID',
 'trip_distance']

In [141]:
#model = ...

def model_predict(df):
    # y_pred = model.predict(df)
    y_pred = df.trip_distance * 5
    return y_pred

In [142]:
def apply_model_in_batch(rows):
    df = pd.DataFrame(rows, columns=columns)
    predictions = model_predict(df)
    df['predicted_duration'] = predictions
    
    for row in df.itertuples():
        yield row
    

In [149]:
df_predicts = duration_rdd \
    .mapPartitions(apply_model_in_batch) \
    .toDF() \
    .drop('Index')

In [151]:
df_predicts.select('predicted_duration').show()

+------------------+
|predicted_duration|
+------------------+
|              43.0|
|              25.8|
|             81.35|
|               7.1|
|              23.1|
|             13.35|
|               3.2|
|              2.15|
|6.8999999999999995|
|               5.0|
|               7.1|
|41.150000000000006|
|             33.55|
|               0.0|
|             41.25|
|7.1499999999999995|
|              51.2|
|              3.05|
| 8.100000000000001|
|17.150000000000002|
+------------------+
only showing top 20 rows

